# Fraud Accounts Life Span Data Collection

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from scipy import stats
get_ipython().magic(u'config IPCompleter.greedy=True')

sns.set(color_codes=True)

In [ ]:
## Amazon AWS Redshift Connection
from contextlib import closing


import psycopg2
import pandas as pd
import simplejson

DEFAULT_DB = ''
DEFAULT_HOST = ''
DEFAULT_PORT = 123


class PsycopgConnector:
    '''
    A database connector that uses Psycopg to connect to Redshift.

    How to play:

        psy_conn = PsycopgConnector(username, password)
        df = psy_conn.run_query(sql=sql, return_data=True)

    NOTE: This class commits queries to redshift if return_data=False.
    This means INSERT, DROP, TRUNCATE, etc. all work against the DB.
    '''

    def __init__(
        self,
        username=None,
        password=None,
        db=DEFAULT_DB,
        host=DEFAULT_HOST,
        port=DEFAULT_PORT,
    ):

        self.db = DEFAULT_DB
        self.host = DEFAULT_HOST
        self.port = DEFAULT_PORT

        self.username = username
        self.password = password

    def _get_connection(self):

        self.conn = psycopg2.connect(
            dbname=self.db,
            user=self.username,
            password=self.password,
            host=self.host,
            port=self.port
        )

        return self.conn

    def run_query(self, sql, return_data=False):

        with closing(self._get_connection()) as conn:
            with conn, conn.cursor() as cur:
                if return_data:
                    return pd.read_sql(sql=sql, con=conn)
                else:
                    cur.execute(sql)
                    

# Read the Redshift's credentials file 
with open("redshift_creds.json.nogit") as fh:
    creds = simplejson.loads(fh.read())
    
username = creds.get("user_name")
password = creds.get("password")

pig = PsycopgConnector(username, password)

username = creds.get("user_name")
password = creds.get("password")

In [ ]:
# Test the connection
query = "SELECT * FROM report_systems LIMIT 10"
df = pig.run_query(query, return_data=True)
df.head(2)

In [ ]:
## Fraud Accounts Lifespan Data Collection
sql_fraud_account_lifespan = '''fraud account lifespan sql'''

# Import as dataframe from redshift
df_fraud_account_lifespan = pig.run_query(sql_fraud_account_lifespan, return_data=True)


In [ ]:
## Test data
df_fraud_account_lifespan.tail()

In [ ]:
# Export as csv file
df_fraud_account_lifespan.to_csv("export_path.csv", sep="\t", index=False)